### Add Policies to the Execution Role 
* In this sample code, we are going to use several AWS services. Therefore we have to add policies to the notebook execution role. 
* Regarding to role and policy, please refer to documents [1](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) and [2](https://docs.aws.amazon.com/IAM/latest/UserGuide/access.html)



In [1]:
import boto3
from sagemaker import get_execution_role

role_name = get_execution_role().split('/')[-1]

iam = boto3.client("iam")

print(role_name)
policy_arns = ["arn:aws:iam::aws:policy/AmazonSQSFullAccess", 
               "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
               "arn:aws:iam::aws:policy/AmazonAPIGatewayAdministrator",
               "arn:aws:iam::aws:policy/AmazonSNSFullAccess",
               "arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess",
               "arn:aws:iam::aws:policy/AWSLambda_FullAccess"]

for p in policy_arns: 
    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = p
    )



AmazonSageMaker-ExecutionRole-20210702T211675


### Alternate Docker Storage Location 

* docker overlay directory usually will occupy large amount of disk space, change the location to EBS volume 

In [5]:
!rm -r /home/ec2-user/SageMaker/docker_disk/docker

rm: descend into write-protected directory ‘/home/ec2-user/SageMaker/docker_disk/docker’? ^C


In [14]:
!sudo rm -r /home/ec2-user/SageMaker/docker_disk/

rm: descend into write-protected directory ‘/home/ec2-user/SageMaker/docker_disk/docker’? ^C


In [10]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [12]:
%%bash 
sudo service docker stop
mkdir ~/SageMaker/docker_disk
sudo mv /var/lib/docker ~/SageMaker/docker_disk/
sudo ln -s  ~/SageMaker/docker_disk/docker/ /var/lib/
sudo service docker start

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


In [3]:
!sudo docker system prune -a -f

Deleted Networks:
sagemaker-local

Deleted Images:
untagged: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04
untagged: 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training@sha256:33bd2f7485f1e486c016a470a5e3e5c9894d2b69734425259fc58fec8054f853
deleted: sha256:7de1db919d5896b81e6ecf3bdc92a7c4049554d06f1bea7c17fd597cb95e7cba
deleted: sha256:fa698ecf8ccade288dbaa8193a56826ea456680be7a417b664150ed9830567af
deleted: sha256:6fffaa06aaee0976590032eabf8e5be578adfbf54a3cdf4c0e4d96ebf4d3e7ec
deleted: sha256:32a9008987dd4e092e79bc0b4c0baae3a2c326f69d97b54611d9bdfc5383a793
deleted: sha256:4fd36020741316d6afc83fbf7fd0ee0df0426396ecd9f0d6981df7e3a25efbcb
deleted: sha256:838a14765633ea2ec2244e101ff5487704c78489f78092d64f2f6fd9455debe6
deleted: sha256:877cb0e75bc371b0d0b321c794ba0306eaee5aa40e0cac46ce0b8d726cf6b461
deleted: sha256:f6a18554dd5bc7477fa8beb79a5c4e85198691253f63a31c78aed2d26b140021
deleted: sha256:bb5377a91e2d47f9a81eac89bf4a28a8cc3b

In [9]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs        31426652       72  31426580   1% /dev
tmpfs           31437352       88  31437264   1% /dev/shm
/dev/xvda1     108305356 89587124  18617984  83% /
/dev/xvdf      515535104 60288308 433189760  13% /home/ec2-user/SageMaker


In [13]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs        31426652       72  31426580   1% /dev
tmpfs           31437352       88  31437264   1% /dev/shm
/dev/xvda1     108305356 89586528  18618580  83% /
/dev/xvdf      515535104 36394092 457083976   8% /home/ec2-user/SageMaker


In [17]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs        31426652       72  31426580   1% /dev
tmpfs           31437352       88  31437264   1% /dev/shm
/dev/xvda1     108305356 89586540  18618568  83% /
/dev/xvdf      515535104 38562876 454915192   8% /home/ec2-user/SageMaker


In [ ]:
base_img='763104351884.dkr.ecr.'$region'.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04'
echo 'base_img:'$base_img
docker pull $base_img
# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} -f Dockerfile  --build-arg BASE_IMG="${base_img}" .  --no-cache
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}


In [20]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00001.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"errorMessage": "An error occurred (ValidationError) when calling the InvokeEndpoint operation: 1 validation error detected: Value at 'body' failed to satisfy constraint: Member must not be null", "errorType": "ValidationError", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 26, in lambda_handler\n    Body=payload)\n", "  File \"/var/runtime/botocore/client.py\", line 357, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/runtime/botocore/client.py\", line 676, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}

In [19]:
from datetime import datetime, timedelta, timezone
print(datetime.now(timezone(timedelta(hours=8))))
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00001.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'
print(datetime.now(timezone(timedelta(hours=8))))

2021-07-16 15:39:49.462996+08:00
{"errorMessage": "An error occurred (ValidationError) when calling the InvokeEndpoint operation: 1 validation error detected: Value at 'body' failed to satisfy constraint: Member must not be null", "errorType": "ValidationError", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 26, in lambda_handler\n    Body=payload)\n", "  File \"/var/runtime/botocore/client.py\", line 357, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/runtime/botocore/client.py\", line 676, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}2021-07-16 15:39:50.839498+08:00
